In [2]:
pip install psycopg[binary]


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import psycopg
import pandas as pd
from psycopg.rows import dict_row
import os

connection = os.environ['hydra_connection']

with psycopg.connect(connection, row_factory=dict_row) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        try:
            # Query the database and obtain data as Python objects.
            cur.execute("SELECT schema_name FROM information_schema.schemata;")
            result = cur.fetchall()
            
            data = pd.DataFrame(result)
            display(data)

        
        except BaseException:
            conn.rollback()
        
        else:
            conn.commit()
        
        finally:
            conn.close()


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,columnar,options,admin,None,True,False,False,False
1,columnar,stripe,admin,None,True,False,True,False
2,columnar,chunk,admin,None,True,False,True,False
3,columnar,chunk_group,admin,None,True,False,True,False
4,columnar,row_mask,admin,None,True,False,False,False
5,public,patient_dim,admin,None,True,False,False,False
6,public,doctor_dim,admin,None,True,False,False,False
7,public,test_dim,admin,None,True,False,False,False
8,public,hospital_events_fact,admin,None,True,False,False,False
